Base de Dados: 

- Usaremos o selenium
- Importante: baixar o webdriver
- recomendações: primeiro baixe o arquivo da base de dados https://docs.google.com/spreadsheets/d/1nKFEgJ5EGvhrzv30Fpp15OFhioSSXpRM/edit?usp=sharing&ouid=107521629934818779354&rtpof=true&sd=true

e depois comece a fazer o codigo


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



# Passo 1: abrir o 
navegador = webdriver.Chrome()
navegador.get("https://www.google.com.br/")
# Passo 2: Navegar sobre ele , pesquisar oq queres ou outras coisas , no caso cotacao do dolar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do dolar")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)
# Passo 3: Navegar sobre ele , pesquisar oq queres ou outras coisas , no caso cotacao do euro
navegador.get("https://www.google.com.br/")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do euro")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)
# Passo 4: Navegar sobre ele , pesquisar oq queres ou outras coisas , no caso cotacao do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)
navegador.quit()

5.1728
5.2900673760000005
293.37


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados
- Salvar ela  em alguma pasta

In [ ]:
import pandas as pd 

df = pd.read_excel(r".xlsx") # caminho de onde voce baixou a base de dados , obs: pode variar

moeda = ["Dólar", "Euro", "Ouro"]
numero = [cotacao_dolar, cotacao_euro, cotacao_ouro]

df = pd.DataFrame()

df["Moeda"] = moeda
df["Cotação"] = numero



df

,Moeda,Cotação
0,Dólar,5.1842999999999995
1,Euro,5.289437604000001
2,Ouro,292.21


- Atualizando os preços e o cálculo do Preço Final

In [ ]:
df.loc[df["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
df.loc[df["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
df.loc[df["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

df

,Moeda,Cotação
0,Dólar,5.1843
1,Euro,5.289438
2,Ouro,292.21


### Agora vamos exportar a nova base de preços atualizada

In [ ]:
df.to_excel(r".xlsx", index=False)# o caminho deve ser o mesmo 
#  nome do arquivo tem ser diferente do original

### Mandar email


In [ ]:
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
   
fromaddr = "seu email"
toaddr = "email da pessoa que vai receber"
   
msg = MIMEMultipart() 
msg['From'] = fromaddr 
msg['To'] = toaddr 
msg['Subject'] = "mensagem de titulo"
corpo_email = "mensagem principal"
msg.attach(MIMEText(corpo_email, 'plain')) 
filename = "Nome da pasta que vai querer salvou" # nome do anexo que vai ficar pro email que enviar
attachment = open(r".xlsx", "rb")    # pasta aonde esta localizado o novo arquivo , ja atualizado
p = MIMEBase('application', 'octet-stream') 
p.set_payload((attachment).read()) 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
msg.attach(p) 
s = smtplib.SMTP('smtp.gmail.com', 587) 
s.starttls() 
s.login(fromaddr, "sua senha") 
text = msg.as_string() 
s.sendmail(fromaddr, toaddr, text) 
s.quit() 

(221,
 b'2.0.0 closing connection kv7-20020a056870fb8700b0010e81e27b99sm953097oab.17 - gsmtp')